In [1]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 기본 directory 설정
import os
os.chdir('/content/drive/MyDrive/Monthly_Workout')

In [3]:
# 모듈 불러오기
import random
import pandas as pd
import numpy as np
from tqdm import tqdm

In [4]:
# 데이터 불러오기
path = './' # 기본 directory 경로에 추가 할 경로

train = pd.read_csv(path + 'train_features.csv')
train_labels = pd.read_csv(path + 'train_labels.csv')
test = pd.read_csv(path + 'test_features.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

In [5]:
act_list=train.iloc[:,2:].columns
acc_list=['acc_x','acc_y','acc_z']
gy_list=['gy_x','gy_y','gy_z']
act_list

Index(['acc_x', 'acc_y', 'acc_z', 'gy_x', 'gy_y', 'gy_z'], dtype='object')

In [6]:
from sklearn.preprocessing import StandardScaler
# Scaling
scaler=StandardScaler()
train[act_list]=scaler.fit_transform(train[act_list])
test[act_list]=scaler.transform(test[act_list])
train

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,2.307314,-0.029939,-0.432104,0.011776,-0.410859,-0.461007
1,0,1,2.421086,-0.063321,-0.504058,0.026484,-0.528052,-0.365459
2,0,2,2.444664,-0.056749,-0.654199,-0.037962,-0.596041,-0.366767
3,0,3,2.428612,-0.116782,-0.573407,0.066099,-0.725311,-0.401232
4,0,4,2.439475,-0.044220,-0.588886,0.091969,-0.784094,-0.408413
...,...,...,...,...,...,...,...,...
1874995,3124,595,-0.367432,-0.845648,0.503707,-0.461266,-1.413128,-1.092639
1874996,3124,596,-0.326315,-0.845833,0.578875,-0.474109,-1.382884,-1.097377
1874997,3124,597,-0.300794,-0.859728,0.652722,-0.436694,-1.341672,-1.136169
1874998,3124,598,-0.253120,-0.886873,0.673002,-0.367051,-1.346630,-1.148831


In [7]:
# acc 데이터와 gy 데이터로 분할
def sensor_split(data):
    X_acc = []
    X_gy = []

    for i in tqdm(data['id'].unique()):
        temp_acc = np.array(data[data['id'] == i].loc[:,acc_list])
        temp_gy = np.array(data[data['id'] == i].loc[:,gy_list])
        X_acc.append(temp_acc)
        X_gy.append(temp_gy)
      
    X_acc = np.array(X_acc).reshape(-1,600,3)
    X_gy = np.array(X_gy).reshape(-1,600,3)

    return X_acc, X_gy

In [8]:
# acc, gy 데이터 분할
X_train_mod=pd.merge(train,train_labels,how='left',on='id')
X_train_acc, X_train_gy= sensor_split(X_train_mod)

X_train_acc.shape, X_train_gy.shape

100%|██████████| 3125/3125 [00:15<00:00, 207.62it/s]


((3125, 600, 3), (3125, 600, 3))

In [9]:
# 데이터 증강

def Permutation(X, nPerm=4, minSegLength=10):
    X_new = np.zeros(X.shape)
    idx = np.random.permutation(nPerm)
    bWhile = True
    while bWhile == True:
        segs = np.zeros(nPerm+1, dtype=int)
        segs[1:-1] = np.sort(np.random.randint(minSegLength, X.shape[0]-minSegLength, nPerm-1))
        segs[-1] = X.shape[0]
        if np.min(segs[1:]-segs[0:-1]) > minSegLength:
            bWhile = False
    pp = 0
    for ii in range(nPerm):
        x_temp = X[segs[idx[ii]]:segs[idx[ii]+1],:]
        X_new[pp:pp+len(x_temp),:] = x_temp
        pp += len(x_temp)
    return (X_new)

def aug(data, uid, shift):
    shift_data = np.roll(data[uid], shift, axis=0)
    return shift_data
def Rolling(data):
    aug_data=[]
    for i in range(data.shape[0]):
        temp=list((aug(data,i,int(random.random()*600))))
        aug_data.append(temp)
    return np.array(aug_data)

In [10]:
# 데이터 증강 (반복하고 싶은 만큼 조정)
X_train_acc_temp = X_train_acc.copy()
X_train_gy_temp = X_train_gy.copy()

y_train = train_labels['label']
y_train_total = np.append(y_train, y_train, axis=0)

rep = 3
for i in range(rep):
    X_train_acc_roll = Rolling(X_train_acc_temp)
    X_train_acc_rp = Permutation(Rolling(X_train_acc_temp)) # rolling + permutation

    X_train_gy_roll = Rolling(X_train_gy_temp)
    X_train_gy_rp = Permutation(Rolling(X_train_gy_temp)) # rolling + permutation

    # 증강시킨 데이터 원래 데이터에 추가
    X_train_acc = np.append(X_train_acc, X_train_acc_roll, axis=0)
    X_train_acc = np.append(X_train_acc, X_train_acc_rp, axis=0)

    X_train_gy = np.append(X_train_gy, X_train_gy_roll, axis=0)
    X_train_gy = np.append(X_train_gy, X_train_gy_rp, axis=0)

    y_train_total = np.append(y_train_total, y_train, axis=0)
    if i != (rep-1): # 마지막 한 번 제외
        y_train_total = np.append(y_train_total, y_train, axis=0)
        
X_train_acc.shape, X_train_gy.shape, y_train_total.shape

((21875, 600, 3), (21875, 600, 3), (21875,))

In [11]:
# np array 형태를 dataframe 으로 변환
acc = [e for sl in X_train_acc for e in sl]

df_report = np.stack(acc, axis = 0)
df_acc = pd.DataFrame(df_report, columns= ['acc_x', 'acc_y', 'acc_z']) 

df_acc

,acc_x,acc_y,acc_z
0,2.307314,-0.029939,-0.432104
1,2.421086,-0.063321,-0.504058
2,2.444664,-0.056749,-0.654199
3,2.428612,-0.116782,-0.573407
4,2.439475,-0.044220,-0.588886
...,...,...,...
13124995,0.004076,-1.234808,0.725889
13124996,0.001228,-1.173413,0.699010
13124997,-0.040607,-1.122513,0.620188
13124998,-0.104412,-1.065670,0.636435


In [12]:
# gy
gy = [e for sl in X_train_gy for e in sl]

df_report = np.stack(gy, axis = 0)
df_gy = pd.DataFrame(df_report, columns= ['gy_x', 'gy_y', 'gy_z']) 

df_gy

,gy_x,gy_y,gy_z
0,0.011776,-0.410859,-0.461007
1,0.026484,-0.528052,-0.365459
2,-0.037962,-0.596041,-0.366767
3,0.066099,-0.725311,-0.401232
4,0.091969,-0.784094,-0.408413
...,...,...,...
13124995,0.842112,0.233547,-0.630914
13124996,0.875561,0.185881,-0.784822
13124997,0.837425,0.171033,-1.010118
13124998,0.704968,0.243441,-1.054380


In [13]:
# acc, gy 데이터프레임 병합
df_aug_result = pd.concat([df_acc, df_gy], axis = 1)

train = df_aug_result
train

,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,2.307314,-0.029939,-0.432104,0.011776,-0.410859,-0.461007
1,2.421086,-0.063321,-0.504058,0.026484,-0.528052,-0.365459
2,2.444664,-0.056749,-0.654199,-0.037962,-0.596041,-0.366767
3,2.428612,-0.116782,-0.573407,0.066099,-0.725311,-0.401232
4,2.439475,-0.044220,-0.588886,0.091969,-0.784094,-0.408413
...,...,...,...,...,...,...
13124995,0.004076,-1.234808,0.725889,0.842112,0.233547,-0.630914
13124996,0.001228,-1.173413,0.699010,0.875561,0.185881,-0.784822
13124997,-0.040607,-1.122513,0.620188,0.837425,0.171033,-1.010118
13124998,-0.104412,-1.065670,0.636435,0.704968,0.243441,-1.054380


In [14]:
train['acc_t'] = (train['acc_x'] ** 2) + (train['acc_y'] ** 2) + (train['acc_z'] ** 2) ** (1/3)
test['acc_t'] = (test['acc_x'] ** 2) + (test['acc_y'] ** 2) + (test['acc_z'] ** 2) ** (1/3)

train['gy_t'] = (train['gy_x'] ** 2) + (train['gy_y'] ** 2) + (train['gy_z'] ** 2) ** (1/3)
test['gy_t'] = (test['gy_x'] ** 2) + (test['gy_y'] ** 2) + (test['gy_z'] ** 2) ** (1/3)

# Signal 극대화 (peak 캐치 유용)
train['acc_mag'] = (train['acc_x'] ** 2) + (train['acc_y'] ** 2) + (train['acc_z'] ** 2)
test['acc_mag'] = (test['acc_x'] ** 2) + (test['acc_y'] ** 2) + (test['acc_z'] ** 2)

train['gy_mag'] = (train['gy_x'] ** 2) + (train['gy_y'] ** 2) + (train['gy_z'] ** 2)
test['gy_mag'] = (test['gy_x'] ** 2) + (test['gy_y'] ** 2) + (test['gy_z'] ** 2)

In [15]:
# vector
train['acc_vector'] = np.sqrt((train['acc_x'] ** 2) +(train['acc_y'] ** 2)+(train['acc_z'] ** 2))
test['acc_vector'] = np.sqrt((test['acc_x'] ** 2) +(test['acc_y'] ** 2)+(test['acc_z'] ** 2))

train['gy_vector'] = np.sqrt((train['gy_x'] ** 2) +(train['gy_y'] ** 2)+(train['gy_z'] ** 2))
test['gy_vector'] = np.sqrt((test['gy_x'] ** 2) +(test['gy_y'] ** 2)+(test['gy_z'] ** 2))

train['acc_YZvector'] = np.sqrt((train['acc_y'] ** 2) +(train['acc_z'] ** 2))
test['acc_YZvector'] = np.sqrt((test['acc_y'] ** 2) +(test['acc_z'] ** 2))

train['gy_YZvector'] = np.sqrt((train['gy_y'] ** 2) +(train['gy_z'] ** 2))
test['gy_YZvector'] = np.sqrt((test['gy_y'] ** 2) +(test['gy_z'] ** 2))

train['acc_XYvector'] = np.sqrt((train['acc_x'] ** 2) +(train['acc_y'] ** 2))
test['acc_XYvector'] = np.sqrt((test['acc_x'] ** 2) +(test['acc_y'] ** 2))

train['gy_XYvector'] = np.sqrt((train['gy_x'] ** 2) +(train['gy_y'] ** 2))
test['gy_XYvector'] = np.sqrt((test['gy_x'] ** 2) +(test['gy_y'] ** 2))

train['acc_XZvector'] = np.sqrt((train['acc_x'] ** 2) +(train['acc_z'] ** 2))
test['acc_XZvector'] = np.sqrt((test['acc_x'] ** 2) +(test['acc_z'] ** 2))

train['gy_XZvector'] = np.sqrt((train['gy_x'] ** 2) +(train['gy_z'] ** 2))
test['gy_XZvector'] = np.sqrt((test['gy_x'] ** 2) +(test['gy_z'] ** 2))

# 자이로스코프 무게중심
train['gy_Centerofgravity'] = (train['gy_x']+train['gy_y']+train['gy_z'])/3
test['gy_Centerofgravity'] = (test['gy_x']+test['gy_y']+test['gy_z'])/3

# roll & pitch
train['roll'] = np.arctan(train['acc_y']/np.sqrt(train['acc_x'] ** 2 + train['acc_z'] ** 2))
test['roll'] = np.arctan(test['acc_y']/np.sqrt(test['acc_x'] ** 2 + test['acc_z'] ** 2))

train['pitch'] = np.arctan(train['acc_x']/np.sqrt(train['acc_y'] ** 2 + train['acc_z'] ** 2))
test['pitch'] = np.arctan(test['acc_x']/np.sqrt(test['acc_y'] ** 2 + test['acc_z'] ** 2))

In [16]:
from math import pi

train['math_roll'] = np.arctan(- train['acc_x']/np.sqrt(train['acc_y'] ** 2 + train['acc_z'] ** 2)) * (180/pi)
test['math_roll'] = np.arctan(- test['acc_x']/np.sqrt(test['acc_y'] ** 2 + test['acc_z'] ** 2)) * (180/pi)

train['math_pitch'] = np.arctan(train['acc_y']/np.sqrt(train['acc_x'] ** 2 + train['acc_z'] ** 2)) * (180/pi)
test['math_pitch'] = np.arctan(test['acc_y']/np.sqrt(test['acc_x'] ** 2 + test['acc_z'] ** 2)) * (180/pi)

train['gy_roll'] = np.arctan(train['gy_y']/np.sqrt(train['gy_x'] ** 2 + train['gy_z'] ** 2))
test['gy_roll'] = np.arctan(test['gy_y']/np.sqrt(test['gy_x'] ** 2 + test['gy_z'] ** 2))

train['gy_pitch'] = np.arctan(train['gy_x']/np.sqrt(train['gy_y'] ** 2 + train['gy_z'] ** 2))
test['gy_pitch'] = np.arctan(test['gy_x']/np.sqrt(test['gy_y'] ** 2 + test['gy_z'] ** 2))

train['gy_math_roll'] = np.arctan(- train['gy_x']/np.sqrt(train['gy_y'] ** 2 + train['gy_z'] ** 2)) * (180/pi)
test['gy_math_roll'] = np.arctan(- test['gy_x']/np.sqrt(test['gy_y'] ** 2 + test['gy_z'] ** 2)) * (180/pi)

train['gy_math_pitch'] = np.arctan(train['gy_y']/np.sqrt(train['gy_x'] ** 2 + train['gy_z'] ** 2)) * (180/pi)
test['gy_math_pitch'] = np.arctan(test['gy_y']/np.sqrt(test['gy_x'] ** 2 + test['gy_z'] ** 2)) * (180/pi)

print(train.shape)
train

(13125000, 27)


,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_t,gy_t,acc_mag,gy_mag,acc_vector,gy_vector,acc_YZvector,gy_YZvector,acc_XYvector,gy_XYvector,acc_XZvector,gy_XZvector,gy_Centerofgravity,roll,pitch,math_roll,math_pitch,gy_roll,gy_pitch,gy_math_roll,gy_math_pitch
0,2.307314,-0.029939,-0.432104,0.011776,-0.410859,-0.461007,5.896150,0.765711,5.511308,0.381471,2.347617,0.617633,0.433140,0.617521,2.307508,0.411027,2.347427,0.461158,-0.286697,-0.012753,1.385231,-79.367910,-0.730704,-0.727781,0.019068,-1.092507,-41.698765
1,2.421086,-0.063321,-0.504058,0.026484,-0.528052,-0.365459,6.499033,0.790703,6.119743,0.413101,2.473812,0.642729,0.508020,0.642183,2.421914,0.528716,2.473001,0.366417,-0.289009,-0.025599,1.363966,-78.149478,-1.466742,-0.964174,0.041217,-2.361573,-55.243076
2,2.444664,-0.056749,-0.654199,-0.037962,-0.596041,-0.366767,6.733202,0.869088,6.407580,0.491224,2.531320,0.700874,0.656655,0.699845,2.445323,0.597249,2.530684,0.368727,-0.333590,-0.022421,1.308383,-74.964812,-1.284609,-1.016793,-0.054190,3.104851,-58.257969
3,2.428612,-0.116782,-0.573407,0.066099,-0.725311,-0.401232,6.601997,1.074443,6.240592,0.691433,2.498118,0.831524,0.585178,0.828893,2.431419,0.728317,2.495386,0.406641,-0.353481,-0.046765,1.334352,-76.452715,-2.679451,-1.059819,0.079576,-4.559348,-60.723136
4,2.439475,-0.044220,-0.588886,0.091969,-0.784094,-0.408413,6.655560,1.173732,6.299780,0.790064,2.509936,0.888855,0.590543,0.884085,2.439876,0.789470,2.509547,0.418640,-0.366846,-0.017619,1.333287,-76.391740,-1.009478,-1.080386,0.103655,-5.938975,-61.901532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13124995,0.004076,-1.234808,0.725889,0.842112,0.233547,-0.630914,2.332461,1.499305,2.051682,1.161749,1.432369,1.077845,1.432364,0.672753,1.234815,0.873897,0.725900,1.052238,0.148248,-1.039347,0.002846,-0.163061,-59.550218,0.218412,0.896734,-51.379068,12.514103
13124996,0.001228,-1.173413,0.699010,0.875561,0.185881,-0.784822,2.164528,1.651998,1.865513,1.417105,1.365838,1.190422,1.365837,0.806534,1.173413,0.895075,0.699011,1.175820,0.092207,-1.033539,0.000899,-0.051498,-59.217429,0.156789,0.826411,-47.349889,8.983320
13124997,-0.040607,-1.122513,0.620188,0.837425,0.171033,-1.010118,1.988932,1.737267,1.646318,1.750872,1.283089,1.323205,1.282447,1.024496,1.123247,0.854712,0.621516,1.312105,-0.000553,-1.065130,-0.031653,1.813574,-61.027458,0.129620,0.685262,-39.262634,7.426654
13124998,-0.104412,-1.065670,0.636435,0.704968,0.243441,-1.054380,1.886448,1.592176,1.551604,1.667960,1.245634,1.291495,1.241250,1.082118,1.070773,0.745817,0.644943,1.268344,-0.035324,-1.026562,-0.083921,4.808310,-58.817684,0.189630,0.577408,-33.083062,10.864986


In [17]:
# 모델 생성
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [18]:
len_features = train.shape[1] # feature 갯수
X=tf.reshape(np.array(train.iloc[:,0:]),[-1, 600, len_features])
X.shape

TensorShape([21875, 600, 27])

In [19]:
trainX = X
trainX.shape

TensorShape([21875, 600, 27])

In [20]:
y = tf.keras.utils.to_categorical(y_train_total) 
y.shape

(21875, 61)

In [21]:
#가벼운 모델 생성
model = Sequential()
model.add(LSTM(32, input_shape=(600,len_features)))
model.add(Dense(128, activation='relu'))
model.add(Dense(61, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
model.fit(X,y, epochs=30, batch_size=128, validation_split=0.2)

Epoch 1/30
137/137 [==============================] - 7s 35ms/step - loss: 2.8309 - accuracy: 0.4709 - val_loss: 2.6619 - val_accuracy: 0.4859
Epoch 2/30
137/137 [==============================] - 4s 32ms/step - loss: 2.5402 - accuracy: 0.4886 - val_loss: 2.6222 - val_accuracy: 0.4843
Epoch 3/30
137/137 [==============================] - 4s 32ms/step - loss: 2.4519 - accuracy: 0.4919 - val_loss: 2.6033 - val_accuracy: 0.4821
Epoch 4/30
137/137 [==============================] - 4s 31ms/step - loss: 2.3841 - accuracy: 0.5003 - val_loss: 2.6070 - val_accuracy: 0.4848
Epoch 5/30
137/137 [==============================] - 4s 32ms/step - loss: 2.3274 - accuracy: 0.5051 - val_loss: 2.6092 - val_accuracy: 0.4855
Epoch 6/30
137/137 [==============================] - 4s 31ms/step - loss: 2.2773 - accuracy: 0.5129 - val_loss: 2.6195 - val_accuracy: 0.4857
Epoch 7/30
137/137 [==============================] - 4s 32ms/step - loss: 2.2281 - accuracy: 0.5190 - val_loss: 2.6267 - val_accuracy: 0.4896

In [23]:
test_X=tf.reshape(np.array(test.iloc[:,2:]),[-1, 600, len_features])
test_X.shape

TensorShape([782, 600, 27])

In [24]:
prediction=model.predict(test_X)

In [25]:
prediction

array([[0.0009877 , 0.00138971, 0.00088044, ..., 0.00879483, 0.00133037,
        0.00963764],
       [0.00047828, 0.0010316 , 0.00024828, ..., 0.00107526, 0.00024785,
        0.00402425],
       [0.00101186, 0.05368865, 0.00015159, ..., 0.00193887, 0.00011232,
        0.00316479],
       ...,
       [0.01424176, 0.00440363, 0.0011936 , ..., 0.00714275, 0.00255643,
        0.00423322],
       [0.00473591, 0.04462571, 0.00249913, ..., 0.00602313, 0.0067474 ,
        0.0068358 ],
       [0.00071384, 0.00122978, 0.00710981, ..., 0.00018735, 0.00151934,
        0.00200411]], dtype=float32)

In [26]:
prediction.shape

(782, 61)

In [27]:
submission

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,3125,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3126,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3127,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3128,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,3129,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3902,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
778,3903,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
779,3904,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
780,3905,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:
submission.iloc[:,1:]=prediction

In [29]:
submission

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,3125,0.000988,0.001390,0.000880,0.000981,0.015534,0.000625,0.001529,0.001028,0.004414,0.172855,0.023376,0.122877,0.013512,0.039068,0.036461,0.000450,0.000870,0.000201,0.000869,0.000076,0.000411,0.003521,0.002714,0.005232,0.028913,0.001742,0.113287,0.000903,0.005182,0.001090,0.100945,0.141407,0.006884,0.003727,0.002298,0.000430,0.004606,0.001943,0.001401,0.001065,0.001247,0.000261,0.001564,0.002557,0.001219,0.003850,0.000360,0.005940,0.003644,0.003959,0.004015,0.012274,0.067958,0.000691,0.001166,0.001001,0.001623,0.001226,0.008795,0.001330,0.009638
1,3126,0.000478,0.001032,0.000248,0.004593,0.003718,0.000648,0.000287,0.000171,0.002706,0.003171,0.001666,0.002333,0.000181,0.005084,0.000582,0.003467,0.002734,0.000533,0.000557,0.000574,0.002443,0.001966,0.001031,0.007161,0.003614,0.001171,0.893442,0.002865,0.001068,0.002699,0.000981,0.000248,0.007927,0.002377,0.001304,0.001721,0.003632,0.000236,0.000228,0.000603,0.000098,0.000781,0.000431,0.001491,0.000130,0.003143,0.000039,0.001411,0.001907,0.002002,0.002369,0.005073,0.000154,0.000359,0.000101,0.001308,0.002182,0.000194,0.001075,0.000248,0.004024
2,3127,0.001012,0.053689,0.000152,0.000037,0.000047,0.000513,0.001115,0.088407,0.020891,0.000406,0.000914,0.001972,0.000299,0.000082,0.000354,0.000921,0.001362,0.000047,0.001689,0.000005,0.000118,0.000356,0.000883,0.000109,0.001288,0.000074,0.200481,0.000920,0.054900,0.006066,0.000939,0.000003,0.002553,0.009015,0.001131,0.000067,0.002679,0.005835,0.022675,0.009922,0.002640,0.000059,0.010411,0.076455,0.154130,0.215320,0.001016,0.006253,0.017835,0.007831,0.000586,0.001507,0.001806,0.000027,0.004517,0.000035,0.000339,0.000087,0.001939,0.000112,0.003165
3,3128,0.001907,0.002335,0.000086,0.000226,0.006263,0.001095,0.000499,0.002786,0.005567,0.003881,0.006633,0.005621,0.015816,0.001506,0.018549,0.000563,0.000273,0.000218,0.001184,0.000128,0.000196,0.002316,0.007560,0.003156,0.016183,0.000825,0.659746,0.000266,0.006159,0.001924,0.004814,0.013271,0.016018,0.004078,0.005494,0.000295,0.000716,0.000789,0.001944,0.000747,0.000863,0.000278,0.005660,0.003166,0.009839,0.007733,0.000201,0.001848,0.023613,0.003955,0.012102,0.001871,0.019189,0.000116,0.001592,0.000112,0.001658,0.010341,0.007537,0.003542,0.063149
4,3129,0.008362,0.003262,0.002890,0.002684,0.004326,0.000722,0.001920,0.001159,0.000925,0.012213,0.004724,0.003688,0.000188,0.000480,0.003350,0.000552,0.001325,0.001156,0.000900,0.000124,0.000094,0.002263,0.008199,0.001590,0.002611,0.000517,0.839962,0.001568,0.001090,0.000588,0.004938,0.003579,0.003648,0.003021,0.005869,0.001207,0.006810,0.003833,0.000942,0.001152,0.000357,0.000467,0.001048,0.000843,0.000390,0.000635,0.000383,0.001282,0.005287,0.008117,0.017584,0.001215,0.000337,0.000187,0.000051,0.003714,0.001293,0.000370,0.003415,0.001361,0.003231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3902,0.018849,0.004283,0.000151,0.000604,0.037151,0.019490,0.000268,0.019616,0.002101,0.002597,0.000810,0.000879,0.000154,0.000576,0.009643,0.005399,0.006810,0.017752,0.026330,0.002179,0.002755,0.003078,0.033059,0.004220,0.001456,0.036097,0.569781,0.000747,0.001061,0.000138,0.002412,0.003530,0.003221,0.011541,0.051115,0.008696,0.001533,0.000755,0.001308,0.000116,0.007793,0.002143,0.001012,0.001305,0.000452,0.000874,0.006426,0.001265,0.000672,0.007977,0.010578,0.000358,0.000027,0.000281,0.000182,0.001152,0.007046,0.002748,0.000309,0.033528,0.001613
778,3903,0.001643,0.012826,0.000530,0.000122,0.002618,0.012269,0.000009,0.090862,0.004309,0.001733,0.000894,0.005020,0.000047,0.000586,0.001971,0.007569,0.000713,0.000760,0.020722,0.000004,0.020968,0.002643,0.008641,0.002051,0.003116

In [30]:
submission.to_csv('LSTM_base(jun).csv', index=False)